In [ ]:
import pandas as pd
import numpy as np

import synapse_utils
import os

from deriva_common import HatracStore, ErmrestCatalog, get_credential

import IPython.core.debugger

import sys
sys.path
sys.path.extend(['/Users/carl/Repos/synapse_datasets'])

Set up debugging and logging.

In [8]:
dbg = IPython.core.debugger.Pdb
from IPython.core.debugger import set_trace


Connect to synapse catalog

Use Deriva datapath library to construct queries to the synapse catalog and retrieve the information about the current set of studies.

In [10]:
study_entities = synapse_utils.get_synapse_studies()
study_uri = study_entities.uri

print('Identified %d studies' % len(study_entities))

Identified 28 studies


Now lets do some local manipulation of the studies in preperation to turning them into PANDAs.  Specifically, we will map the protocol name into the protocol type.

In [11]:
study_list = []

protocol_types = {
    'PrcDsy20160101A': 'aversion',  
    'PrcDsy20170613A': 'conditioned', 
    'PrcDsy20170615A': 'unconditioned', 
    'PrcDsy20170613B': 'control'
}

for i in study_entities:
    if protocol_types[i['Protocol']] == 'aversion':
        i['Type'] = 'learner' if i['Learner'] == True else 'nonlearner'
    else:
        i['Type'] = protocol_types[i['Protocol']]
    study_list.append(i)

In [ ]:
study_list[0]

Now lets map over the list of studies and for each study, retrieve the synapse data and turn into PANDA.

In [ ]:
synapses = [synapse_utils.get_synapses(study) for study in study_list]

Lets set up to do a 3D scatter plot of the synapse data.

In [ ]:
# Plot a study

import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)

def plot_study(study):
    '''
    Create a 3D scatter plot of a study.
    '''
    before_synapses = go.Scatter3d(
        x=study['Before']['X'],
        y=study['Before']['Y'],
        z=study['Before']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.8
        )
    )

    after_synapses = go.Scatter3d(
        x=study['After']['X'],
        y=study['After']['Y'],
        z=study['After']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.8
        )
    )

    data = [before_synapses, after_synapses]

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [ ]:
# Plot synapses from PANDAs....

plot_study(synapses[0])

Use a utility function to dump out all of the studies into a local directory.  
This should eventually be eliminated in favor of just dumping out a BDBag.

In [ ]:
# Dump out the synapses to a local directory

destdir = '/Users/carl/Desktop'
current_dir = os.getcwd()

try:
    os.chdir(destdir)
    synapse_utils.export_synapse_studies(objectstore, study_list)
finally:
    os.chdir(current_dir)


Create a MINID for the data set using the current version of the catalog.

In [86]:
import sys
identifier = '/Users/carl/Repos/identifiers' 

sys.path.extend([identifier])

import deriva_minid
from deriva_common import VersionedCatalog
import minid_client.minid_client_api as mca

vc = VersionedCatalog(study_uri)

config = mca.parse_config(mca.DEFAULT_CONFIG_FILE)
minid_server = config['minid_server']
email = config['email']
code = config['code']
test = True
title = None

minid = deriva_minid.create_catalog_minid(vc, minid_server, email, code, title='Latest synapse data', test=test)

print("Created new identifier: %s" % minid)


checksum path is: /attribute/study:=Zebrafish:Synaptic%20Pair%20Study/pair:=Zebrafish:Image%20Pair%20Study/Behavior:=left(Subject)=(Zebrafish:Behavior:Subject)/$pair/Image:=(pair:Image%201)=(Zebrafish:Image:ID)/Protocol%2520Step:=Synapse:Protocol%20Step/Protocol:=Synapse:Protocol/$study/study:Study,pair:Subject,Behavior:Learned%3F,study:Region%201%20URL,study:Region%202%20URL,Protocol:=Protocol:ID


MINIDError: MINID for this catalog version already exists: ark:/99999/fk49g5xv8h